**Importing libraries**

In [1]:
import requests
import pandas as pd
import time

**Pulling Matches Data**

In [3]:
# Base URLs for the OpenDota API
BASE_URL = "https://api.opendota.com/api"
PUBLIC_MATCHES_URL = f"{BASE_URL}/publicMatches"
MATCH_DETAIL_URL = f"{BASE_URL}/matches/{{match_id}}"

# Function to fetch batches of public matches
def get_public_matches(num_batches=2, delay=1):
    """
    Pulls batches of public matches from OpenDota API.
    num_batches: number of batches to fetch (each batch fetches 50 matches)
    delay: delay in seconds between requests (to avoid hitting rate limits)
    """
    matches = []
    for i in range(num_batches):
        response = requests.get(PUBLIC_MATCHES_URL)
        if response.status_code == 200:
            data = response.json()
            matches.extend(data)
        else:
            print(f"Error fetching matches: {response.status_code}")
        time.sleep(delay)  # Avoid hitting the rate limit
    return matches

# Function to fetch detailed match data (including player statistics)
def get_match_details(match_ids, delay=1):
    """
    Fetches detailed match data for the given match IDs from OpenDota API.
    """
    all_details = []
    for match_id in match_ids:
        url = MATCH_DETAIL_URL.format(match_id=match_id)
        response = requests.get(url)
        if response.status_code == 200:
            match_data = response.json()
            # Flatten player data for each match
            for player in match_data.get("players", []):
                all_details.append({
                    "match_id": match_id,
                    "start_time": match_data.get("start_time"),
                    "duration": match_data.get("duration"),
                    "radiant_win": match_data.get("radiant_win"),
                    "player_slot": player.get("player_slot"),
                    "account_id": player.get("account_id"),
                    "hero_id": player.get("hero_id"),
                    "kills": player.get("kills"),
                    "deaths": player.get("deaths"),
                    "assists": player.get("assists"),
                    "gold_per_min": player.get("gold_per_min"),
                    "xp_per_min": player.get("xp_per_min"),
                    "last_hits": player.get("last_hits"),
                    "denies": player.get("denies"),
                    "gold": player.get("gold"),
                    "hero_damage": player.get("hero_damage"),
                    "hero_healing": player.get("hero_healing")
                })
        else:
            print(f"Error fetching match {match_id}: {response.status_code}")
        time.sleep(delay)  # Avoid hitting the rate limit
    return all_details

# ---- Main Execution ----
if __name__ == "__main__":
    print("Fetching public matches...")
    # Fetch 3 batches (150 matches)
    public_matches = get_public_matches(num_batches=3)  
    match_ids = [m["match_id"] for m in public_matches]

    print("Fetching detailed match data...")
    match_details = get_match_details(match_ids)

    # Convert the data to a pandas DataFrame
    df = pd.DataFrame(match_details)

    # Save the data to a CSV file
    csv_path = r"C:\Users\Ellen\Desktop\OpenDota Data\opendota_matches.csv" 
    df.to_csv(csv_path, index=False)
    print(f"Saved {len(df)} rows to {csv_path}")

Fetching public matches...
Fetching detailed match data...
Error fetching match 8390471517: 500
Saved 2990 rows to C:\Users\Ellen\Desktop\OpenDota Data\opendota_matches.csv


**Re-pulling data(handling errors)**

In [5]:
# Base URLs for the OpenDota API
BASE_URL = "https://api.opendota.com/api"
PUBLIC_MATCHES_URL = f"{BASE_URL}/publicMatches"
MATCH_DETAIL_URL = f"{BASE_URL}/matches/{{match_id}}"

# Function to fetch batches of public matches
def get_public_matches(num_batches=2, delay=1):
    """
    Pulls batches of public matches from OpenDota API.
    num_batches: number of batches to fetch (each batch fetches 50 matches)
    delay: delay in seconds between requests (to avoid hitting rate limits)
    """
    matches = []
    for i in range(num_batches):
        response = requests.get(PUBLIC_MATCHES_URL)
        if response.status_code == 200:
            data = response.json()
            matches.extend(data)
        else:
            print(f"Error fetching matches: {response.status_code}")
        time.sleep(delay)  # Avoid hitting the rate limit
    return matches

# Function to fetch detailed match data (including player statistics)
def get_match_details(match_ids, delay=1):
    """
    Fetches detailed match data for the given match IDs from OpenDota API.
    Skips any matches that return an error (status code 500).
    """
    all_details = []
    for match_id in match_ids:
        url = MATCH_DETAIL_URL.format(match_id=match_id)
        response = requests.get(url)
        if response.status_code == 200:
            match_data = response.json()
            # Flatten player data for each match
            for player in match_data.get("players", []):
                all_details.append({
                    "match_id": match_id,
                    "start_time": match_data.get("start_time"),
                    "duration": match_data.get("duration"),
                    "radiant_win": match_data.get("radiant_win"),
                    "player_slot": player.get("player_slot"),
                    "account_id": player.get("account_id"),
                    "hero_id": player.get("hero_id"),
                    "kills": player.get("kills"),
                    "deaths": player.get("deaths"),
                    "assists": player.get("assists"),
                    "gold_per_min": player.get("gold_per_min"),
                    "xp_per_min": player.get("xp_per_min"),
                    "last_hits": player.get("last_hits"),
                    "denies": player.get("denies"),
                    "gold": player.get("gold"),
                    "hero_damage": player.get("hero_damage"),
                    "hero_healing": player.get("hero_healing")
                })
        elif response.status_code == 500:
            print(f"Error fetching match {match_id}: Internal Server Error (500). Skipping this match.")
        else:
            print(f"Error fetching match {match_id}: Status code {response.status_code}. Skipping this match.")
        time.sleep(delay)  # Avoid hitting the rate limit
    return all_details

# ---- Main Execution ----
if __name__ == "__main__":
    print("Fetching public matches...")
    # Fetch 3 batches (150 matches)
    public_matches = get_public_matches(num_batches=3)  
    match_ids = [m["match_id"] for m in public_matches]

    print("Fetching detailed match data...")
    match_details = get_match_details(match_ids)

    # Convert the data to a pandas DataFrame
    df = pd.DataFrame(match_details)

    # Save the data to a CSV file
    csv_path = r"C:\Users\Ellen\Desktop\OpenDota Data\opendota_matches.csv"  
    df.to_csv(csv_path, index=False)
    print(f"Saved {len(df)} rows to {csv_path}")

Fetching public matches...
Fetching detailed match data...
Error fetching match 8390491615: Internal Server Error (500). Skipping this match.
Error fetching match 8390495200: Status code 429. Skipping this match.
Error fetching match 8390495105: Status code 429. Skipping this match.
Error fetching match 8390495014: Status code 429. Skipping this match.
Error fetching match 8390495012: Status code 429. Skipping this match.
Error fetching match 8390494505: Status code 429. Skipping this match.
Error fetching match 8390494417: Status code 429. Skipping this match.
Error fetching match 8390494413: Status code 429. Skipping this match.
Error fetching match 8390494411: Status code 429. Skipping this match.
Error fetching match 8390494410: Status code 429. Skipping this match.
Error fetching match 8390494409: Status code 429. Skipping this match.
Error fetching match 8390494402: Status code 429. Skipping this match.
Error fetching match 8390494400: Status code 429. Skipping this match.
Error 

**Pulling player data for 40 players** 

In [12]:
import requests
import pandas as pd
import time

# Base URL for OpenDota API
BASE_URL = "https://api.opendota.com/api"
PLAYER_MATCHES_URL = f"{BASE_URL}/players/{{account_id}}/matches"

# Your sampled player IDs (already included)
sampled_players = [
    103356886, 88596567, 146532881, 307648521, 29366434, 109362160,
    200135497, 171627276, 103138418, 125774566, 104482314, 127763760,
    175184565, 152883119, 105081622, 91444156, 107386895, 116905783,
    156159900, 103231619, 130747907, 103980011, 110417858, 103226608,
    108304393, 125108963, 146227106, 127499210, 87014496, 182008047,
    139204898, 104496653, 126831383, 88522209, 111654271, 103432006,
    111377393, 121074870, 101423931, 124203827
]

def fetch_player_matches(account_id, num_matches=20, delay=1):
    """
    Fetches recent matches for a player by account_id.
    Handles 429 rate limit errors by waiting and retrying.
    """
    url = PLAYER_MATCHES_URL.format(account_id=account_id)
    params = {"limit": num_matches}
    response = requests.get(url, params=params)
    
    # Handle rate limiting (429)
    if response.status_code == 429:
        retry_after = int(response.headers.get('Retry-After', 60))
        print(f"Rate limit hit for {account_id}. Retrying after {retry_after} seconds...")
        time.sleep(retry_after)
        response = requests.get(url, params=params)

    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error fetching matches for player {account_id}: {response.status_code}")
        return []

# ---- Main Execution ----
all_player_matches = []

print("Fetching player match data...")
for account_id in sampled_players:
    matches = fetch_player_matches(account_id, num_matches=20)
    for match in matches:
        all_player_matches.append({
            "account_id": account_id,
            "match_id": match.get("match_id"),
            "hero_id": match.get("hero_id"),
            "kills": match.get("kills"),
            "deaths": match.get("deaths"),
            "assists": match.get("assists"),
            "gold_per_min": match.get("gold_per_min"),
            "xp_per_min": match.get("xp_per_min"),
            "duration": match.get("duration"),
            "radiant_win": match.get("radiant_win"),
            "start_time": match.get("start_time")
        })
    time.sleep(1)  # Delay between players

# Convert to DataFrame
df_players = pd.DataFrame(all_player_matches)

# Save to CSV
csv_path = r"C:\Users\Ellen\Desktop\OpenDota Data\player_match_data.csv"
df_players.to_csv(csv_path, index=False)
print(f"Saved {len(df_players)} rows to {csv_path}")

Fetching player match data...
Saved 0 rows to C:\Users\Ellen\Desktop\OpenDota Data\player_match_data.csv


In [15]:
# Base URL for OpenDota API
BASE_URL = "https://api.opendota.com/api"
PLAYER_MATCHES_URL = f"{BASE_URL}/players/{{account_id}}/matches"

# Your sampled player IDs (already included)
sampled_players = [
    103356886, 88596567, 146532881, 307648521, 29366434, 109362160,
    200135497, 171627276, 103138418, 125774566, 104482314, 127763760,
    175184565, 152883119, 105081622, 91444156, 107386895, 116905783,
    156159900, 103231619, 130747907, 103980011, 110417858, 103226608,
    108304393, 125108963, 146227106, 127499210, 87014496, 182008047,
    139204898, 104496653, 126831383, 88522209, 111654271, 103432006,
    111377393, 121074870, 101423931, 124203827
]

def fetch_player_matches(account_id, num_matches=20):
    """
    Fetches recent matches for a player by account_id.
    Handles 429 rate limit errors by waiting and retrying.
    """
    url = PLAYER_MATCHES_URL.format(account_id=account_id)
    params = {"limit": num_matches}
    response = requests.get(url, params=params)
    
    # Handle rate limiting (429)
    if response.status_code == 429:
        retry_after = int(response.headers.get('Retry-After', 60))
        print(f"Rate limit hit for {account_id}. Retrying after {retry_after} seconds...")
        time.sleep(retry_after)
        response = requests.get(url, params=params)

    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error fetching matches for player {account_id}: {response.status_code}")
        return []

# ---- Main Execution ----
all_player_matches = []

print("Fetching player match data...")
for i, account_id in enumerate(sampled_players, start=1):
    print(f"Fetching data for player {i} of {len(sampled_players)} (account_id={account_id})...")
    matches = fetch_player_matches(account_id, num_matches=20)
    for match in matches:
        all_player_matches.append({
            "account_id": account_id,
            "match_id": match.get("match_id"),
            "hero_id": match.get("hero_id"),
            "kills": match.get("kills"),
            "deaths": match.get("deaths"),
            "assists": match.get("assists"),
            "gold_per_min": match.get("gold_per_min"),
            "xp_per_min": match.get("xp_per_min"),
            "duration": match.get("duration"),
            "radiant_win": match.get("radiant_win"),
            "start_time": match.get("start_time")
        })
    print(f"Finished player {i}. Matches collected so far: {len(all_player_matches)}")
    time.sleep(1)  # Delay between players

# Convert to DataFrame
df_players = pd.DataFrame(all_player_matches)

# Save to CSV
csv_path = r"C:\Users\Ellen\Desktop\OpenDota Data\player_match_data.csv"
df_players.to_csv(csv_path, index=False)
print(f"Saved {len(df_players)} rows to {csv_path}")

Fetching player match data...
Fetching data for player 1 of 40 (account_id=103356886)...
Finished player 1. Matches collected so far: 0
Fetching data for player 2 of 40 (account_id=88596567)...
Finished player 2. Matches collected so far: 0
Fetching data for player 3 of 40 (account_id=146532881)...
Finished player 3. Matches collected so far: 0
Fetching data for player 4 of 40 (account_id=307648521)...
Finished player 4. Matches collected so far: 0
Fetching data for player 5 of 40 (account_id=29366434)...
Finished player 5. Matches collected so far: 0
Fetching data for player 6 of 40 (account_id=109362160)...
Finished player 6. Matches collected so far: 0
Fetching data for player 7 of 40 (account_id=200135497)...
Finished player 7. Matches collected so far: 0
Fetching data for player 8 of 40 (account_id=171627276)...
Finished player 8. Matches collected so far: 0
Fetching data for player 9 of 40 (account_id=103138418)...
Finished player 9. Matches collected so far: 0
Fetching data for 

**Pull retry for (100 match stats)**

In [20]:
# Base URLs for OpenDota API
BASE_URL = "https://api.opendota.com/api"
MATCH_DETAIL_URL = f"{BASE_URL}/matches/{{match_id}}"

# --- Load your match IDs ---
matches_df = pd.read_csv(r"C:\Users\Ellen\Desktop\OpenDota Data\opendota_matches.csv")
match_ids = matches_df['match_id'].unique()[:500]  # Take first 500 matches

def fetch_match_details(match_id):
    """
    Fetch detailed match data from OpenDota API.
    Handles 429 rate limit errors by waiting and retrying.
    """
    url = MATCH_DETAIL_URL.format(match_id=match_id)
    response = requests.get(url)

    # Handle rate limit (429)
    if response.status_code == 429:
        retry_after = int(response.headers.get('Retry-After', 60))
        print(f"Rate limit hit for match {match_id}. Retrying after {retry_after} seconds...")
        time.sleep(retry_after)
        response = requests.get(url)

    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error fetching match {match_id}: {response.status_code}")
        return None

# --- Collect player stats ---
all_player_data = []
print("Fetching detailed match data...")
for i, match_id in enumerate(match_ids, start=1):
    print(f"Fetching match {i} of {len(match_ids)} (match_id={match_id})...")
    match_data = fetch_match_details(match_id)

    if match_data:
        for player in match_data.get("players", []):
            all_player_data.append({
                "match_id": match_id,
                "account_id": player.get("account_id"),
                "hero_id": player.get("hero_id"),
                "kills": player.get("kills"),
                "deaths": player.get("deaths"),
                "assists": player.get("assists"),
                "gold_per_min": player.get("gold_per_min"),
                "xp_per_min": player.get("xp_per_min"),
                "hero_damage": player.get("hero_damage"),
                "hero_healing": player.get("hero_healing"),
                "duration": match_data.get("duration"),
                "radiant_win": match_data.get("radiant_win"),
                "start_time": match_data.get("start_time")
            })

    time.sleep(1)  # Delay to avoid hitting rate limits

# --- Save to CSV ---
df_players = pd.DataFrame(all_player_data)
csv_path = r"C:\Users\Ellen\Desktop\OpenDota Data\match_details_subset.csv"
df_players.to_csv(csv_path, index=False)
print(f"Saved {len(df_players)} rows to {csv_path}")

Fetching detailed match data...
Fetching match 1 of 101 (match_id=8390507617)...
Fetching match 2 of 101 (match_id=8390507011)...
Fetching match 3 of 101 (match_id=8390506011)...
Fetching match 4 of 101 (match_id=8390503402)...
Fetching match 5 of 101 (match_id=8390502507)...
Fetching match 6 of 101 (match_id=8390502214)...
Fetching match 7 of 101 (match_id=8390502015)...
Fetching match 8 of 101 (match_id=8390500200)...
Fetching match 9 of 101 (match_id=8390499919)...
Fetching match 10 of 101 (match_id=8390499610)...
Fetching match 11 of 101 (match_id=8390499003)...
Fetching match 12 of 101 (match_id=8390498812)...
Fetching match 13 of 101 (match_id=8390498502)...
Fetching match 14 of 101 (match_id=8390498412)...
Fetching match 15 of 101 (match_id=8390498303)...
Fetching match 16 of 101 (match_id=8390497906)...
Fetching match 17 of 101 (match_id=8390497704)...
Fetching match 18 of 101 (match_id=8390497502)...
Fetching match 19 of 101 (match_id=8390497004)...
Fetching match 20 of 101 (m

**Pull remaining 400 match stats**

In [31]:
import os

# File paths
matches_file = r"C:\Users\Ellen\Desktop\OpenDota Data\opendota_matches.csv"
output_file = r"C:\Users\Ellen\Desktop\OpenDota Data\match_details_500matches.csv"

# Base URL for OpenDota API
BASE_URL = "https://api.opendota.com/api"
MATCH_DETAIL_URL = f"{BASE_URL}/matches/{{match_id}}"

# --- Load match IDs ---
all_matches_df = pd.read_csv(matches_file)
match_ids = all_matches_df['match_id'].unique()[:500]  # Top 500 matches

# --- Load existing output (if available) ---
if os.path.exists(output_file):
    existing_df = pd.read_csv(output_file)
    processed_matches = set(existing_df['match_id'].unique())
    print(f"Resuming: {len(processed_matches)} matches already processed, {len(existing_df)} rows in file.")
else:
    existing_df = pd.DataFrame()
    processed_matches = set()
    print("No existing file found. Starting fresh.")

# --- Find remaining matches ---
remaining_match_ids = [mid for mid in match_ids if mid not in processed_matches]
print(f"Remaining matches to fetch: {len(remaining_match_ids)}")

def fetch_match_details(match_id):
    url = MATCH_DETAIL_URL.format(match_id=match_id)
    response = requests.get(url)
    if response.status_code == 429:  # Handle rate limit
        retry_after = int(response.headers.get('Retry-After', 60))
        print(f"Rate limit hit. Waiting {retry_after} seconds...")
        time.sleep(retry_after)
        response = requests.get(url)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error fetching match {match_id}: {response.status_code}")
        return None

# --- Fetch matches ---
all_new_data = []
total_rows = len(existing_df)  # Track row count from existing data

for i, match_id in enumerate(remaining_match_ids, start=1):
    print(f"Fetching match {i} of {len(remaining_match_ids)} (match_id={match_id})...")
    match_data = fetch_match_details(match_id)
    if match_data:
        player_rows = []
        for player in match_data.get("players", []):
            player_rows.append({
                "match_id": match_id,
                "account_id": player.get("account_id"),
                "hero_id": player.get("hero_id"),
                "kills": player.get("kills"),
                "deaths": player.get("deaths"),
                "assists": player.get("assists"),
                "gold_per_min": player.get("gold_per_min"),
                "xp_per_min": player.get("xp_per_min"),
                "hero_damage": player.get("hero_damage"),
                "hero_healing": player.get("hero_healing"),
                "duration": match_data.get("duration"),
                "radiant_win": match_data.get("radiant_win"),
                "start_time": match_data.get("start_time")
            })
        all_new_data.extend(player_rows)
        total_rows += len(player_rows)
        print(f" → Rows so far: {total_rows}")
    time.sleep(1)  # Prevent rate-limit

# --- Combine old + new data ---
if len(existing_df) > 0:
    final_df = pd.concat([existing_df, pd.DataFrame(all_new_data)], ignore_index=True)
else:
    final_df = pd.DataFrame(all_new_data)

# --- Save file ---
final_df.to_csv(output_file, index=False)
print(f"Saved {len(final_df)} rows to {output_file}")
print(f"Matches processed: {len(final_df['match_id'].unique())} of 500")

No existing file found. Starting fresh.
Remaining matches to fetch: 101
Fetching match 1 of 101 (match_id=8390507617)...
 → Rows so far: 10
Fetching match 2 of 101 (match_id=8390507011)...
 → Rows so far: 20
Fetching match 3 of 101 (match_id=8390506011)...
 → Rows so far: 30
Fetching match 4 of 101 (match_id=8390503402)...
 → Rows so far: 40
Fetching match 5 of 101 (match_id=8390502507)...
 → Rows so far: 50
Fetching match 6 of 101 (match_id=8390502214)...
 → Rows so far: 60
Fetching match 7 of 101 (match_id=8390502015)...
 → Rows so far: 70
Fetching match 8 of 101 (match_id=8390500200)...
 → Rows so far: 80
Fetching match 9 of 101 (match_id=8390499919)...
 → Rows so far: 90
Fetching match 10 of 101 (match_id=8390499610)...
 → Rows so far: 100
Fetching match 11 of 101 (match_id=8390499003)...
 → Rows so far: 110
Fetching match 12 of 101 (match_id=8390498812)...
 → Rows so far: 120
Fetching match 13 of 101 (match_id=8390498502)...
 → Rows so far: 130
Fetching match 14 of 101 (match_id=8

In [34]:
# --- Config ---
OUTPUT_FILE = r"C:\Users\Ellen\Desktop\OpenDota Data\opendota_matches_500plus.csv"
NUM_BATCHES = 11   # 11 x 50 = 550 matches
DELAY = 1          # seconds between requests

BASE_URL = "https://api.opendota.com/api"
PUBLIC_MATCHES_URL = f"{BASE_URL}/publicMatches"

all_matches = []

print(f"Fetching {NUM_BATCHES * 50} match IDs from OpenDota...")
for batch in range(NUM_BATCHES):
    response = requests.get(PUBLIC_MATCHES_URL)
    if response.status_code == 200:
        data = response.json()
        all_matches.extend(data)
        print(f"Batch {batch+1}/{NUM_BATCHES}: Collected {len(data)} matches, total so far {len(all_matches)}")
    else:
        print(f"Error fetching batch {batch+1}: {response.status_code}")
    time.sleep(DELAY)

# --- Save to CSV ---
df_matches = pd.DataFrame(all_matches)
df_matches.drop_duplicates(subset=["match_id"], inplace=True)
df_matches.to_csv(OUTPUT_FILE, index=False)

print(f"Saved {len(df_matches)} unique match IDs to {OUTPUT_FILE}")

Fetching 550 match IDs from OpenDota...
Batch 1/11: Collected 100 matches, total so far 100
Batch 2/11: Collected 100 matches, total so far 200
Batch 3/11: Collected 100 matches, total so far 300
Batch 4/11: Collected 100 matches, total so far 400
Batch 5/11: Collected 100 matches, total so far 500
Batch 6/11: Collected 100 matches, total so far 600
Batch 7/11: Collected 100 matches, total so far 700
Batch 8/11: Collected 100 matches, total so far 800
Batch 9/11: Collected 100 matches, total so far 900
Batch 10/11: Collected 100 matches, total so far 1000
Batch 11/11: Collected 100 matches, total so far 1100
Saved 105 unique match IDs to C:\Users\Ellen\Desktop\OpenDota Data\opendota_matches_500plus.csv


**1. Pagination script for 500 matches**

In [44]:
OUTPUT_FILE = r"C:\Users\Ellen\Desktop\OpenDota Data\opendota_matches_500plus.csv"
NUM_BATCHES = 11   # ~550 matches
DELAY = 1

BASE_URL = "https://api.opendota.com/api/publicMatches"

all_matches = []
less_than_match_id = None

print(f"Fetching paginated matches from OpenDota ({NUM_BATCHES} batches)...")
for batch in range(NUM_BATCHES):
    params = {}
    if less_than_match_id:
        params["less_than_match_id"] = less_than_match_id

    response = requests.get(BASE_URL, params=params)
    if response.status_code == 200:
        data = response.json()
        if not data:
            print("No more matches returned by API.")
            break
        all_matches.extend(data)
        less_than_match_id = min(match["match_id"] for match in data)  # move pagination pointer
        print(f"Batch {batch+1}/{NUM_BATCHES}: Collected {len(data)} matches, total so far {len(all_matches)}")
    else:
        print(f"Error fetching batch {batch+1}: {response.status_code}")
    time.sleep(DELAY)

# Deduplicate and save
df_matches = pd.DataFrame(all_matches).drop_duplicates(subset=["match_id"])
df_matches.to_csv(OUTPUT_FILE, index=False)

print(f"Saved {len(df_matches)} unique match IDs to {OUTPUT_FILE}")

Fetching paginated matches from OpenDota (11 batches)...
Batch 1/11: Collected 100 matches, total so far 100
Batch 2/11: Collected 100 matches, total so far 200
Batch 3/11: Collected 100 matches, total so far 300
Batch 4/11: Collected 100 matches, total so far 400
Batch 5/11: Collected 100 matches, total so far 500
Batch 6/11: Collected 100 matches, total so far 600
Batch 7/11: Collected 100 matches, total so far 700
Batch 8/11: Collected 100 matches, total so far 800
Batch 9/11: Collected 100 matches, total so far 900
Batch 10/11: Collected 100 matches, total so far 1000
Batch 11/11: Collected 100 matches, total so far 1100
Saved 1100 unique match IDs to C:\Users\Ellen\Desktop\OpenDota Data\opendota_matches_500plus.csv


**2. Resume-enabled match detail script**
(Resume + Row Counter + New Match File)

In [49]:
# --- File paths ---
matches_file = r"C:\Users\Ellen\Desktop\OpenDota Data\opendota_matches_500plus.csv"
output_file = r"C:\Users\Ellen\Desktop\OpenDota Data\match_details_500matches.csv"

# Base URL for OpenDota API
BASE_URL = "https://api.opendota.com/api"
MATCH_DETAIL_URL = f"{BASE_URL}/matches/{{match_id}}"

# --- Load match IDs ---
all_matches_df = pd.read_csv(matches_file)
match_ids = all_matches_df['match_id'].unique()[:500]

# --- Load existing output (if available) ---
if os.path.exists(output_file):
    existing_df = pd.read_csv(output_file)
    processed_matches = set(existing_df['match_id'].unique())
    print(f"Resuming: {len(processed_matches)} matches already processed, {len(existing_df)} rows in file.")
else:
    existing_df = pd.DataFrame()
    processed_matches = set()
    print("No existing file found. Starting fresh.")

# --- Find remaining matches ---
remaining_match_ids = [mid for mid in match_ids if mid not in processed_matches]
print(f"Remaining matches to fetch: {len(remaining_match_ids)}")

def fetch_match_details(match_id):
    url = MATCH_DETAIL_URL.format(match_id=match_id)
    response = requests.get(url)
    if response.status_code == 429:  # Rate limit handling
        retry_after = int(response.headers.get('Retry-After', 60))
        print(f"Rate limit hit. Waiting {retry_after} seconds...")
        time.sleep(retry_after)
        response = requests.get(url)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error fetching match {match_id}: {response.status_code}")
        return None

# --- Fetch matches ---
all_new_data = []
total_rows = len(existing_df)

for i, match_id in enumerate(remaining_match_ids, start=1):
    print(f"Fetching match {i} of {len(remaining_match_ids)} (match_id={match_id})...")
    match_data = fetch_match_details(match_id)
    if match_data:
        player_rows = []
        for player in match_data.get("players", []):
            player_rows.append({
                "match_id": match_id,
                "account_id": player.get("account_id"),
                "hero_id": player.get("hero_id"),
                "kills": player.get("kills"),
                "deaths": player.get("deaths"),
                "assists": player.get("assists"),
                "gold_per_min": player.get("gold_per_min"),
                "xp_per_min": player.get("xp_per_min"),
                "hero_damage": player.get("hero_damage"),
                "hero_healing": player.get("hero_healing"),
                "duration": match_data.get("duration"),
                "radiant_win": match_data.get("radiant_win"),
                "start_time": match_data.get("start_time")
            })
        all_new_data.extend(player_rows)
        total_rows += len(player_rows)
        print(f" → Rows so far: {total_rows}")
    time.sleep(1)

# --- Combine old + new data ---
if len(existing_df) > 0:
    final_df = pd.concat([existing_df, pd.DataFrame(all_new_data)], ignore_index=True)
else:
    final_df = pd.DataFrame(all_new_data)

# --- Save file ---
final_df.to_csv(output_file, index=False)
print(f"Saved {len(final_df)} rows to {output_file}")
print(f"Matches processed: {len(final_df['match_id'].unique())} of 500")

Resuming: 101 matches already processed, 1010 rows in file.
Remaining matches to fetch: 500
Fetching match 1 of 500 (match_id=8391314512)...
 → Rows so far: 1020
Fetching match 2 of 500 (match_id=8391312415)...
 → Rows so far: 1030
Fetching match 3 of 500 (match_id=8391311716)...
 → Rows so far: 1040
Fetching match 4 of 500 (match_id=8391311417)...
 → Rows so far: 1050
Fetching match 5 of 500 (match_id=8391310813)...
 → Rows so far: 1060
Fetching match 6 of 500 (match_id=8391309804)...
 → Rows so far: 1070
Fetching match 7 of 500 (match_id=8391309707)...
 → Rows so far: 1080
Fetching match 8 of 500 (match_id=8391308918)...
 → Rows so far: 1090
Fetching match 9 of 500 (match_id=8391308906)...
 → Rows so far: 1100
Fetching match 10 of 500 (match_id=8391308717)...
 → Rows so far: 1110
Fetching match 11 of 500 (match_id=8391308506)...
 → Rows so far: 1120
Fetching match 12 of 500 (match_id=8391307704)...
 → Rows so far: 1130
Fetching match 13 of 500 (match_id=8391306714)...
 → Rows so far:

**Script re-run for remaining 499 player match data**

In [52]:
# --- File paths ---
matches_file = r"C:\Users\Ellen\Desktop\OpenDota Data\opendota_matches_500plus.csv"
output_file = r"C:\Users\Ellen\Desktop\OpenDota Data\match_details_500matches.csv"

# Base URL for OpenDota API
BASE_URL = "https://api.opendota.com/api"
MATCH_DETAIL_URL = f"{BASE_URL}/matches/{{match_id}}"

# --- Load match IDs ---
all_matches_df = pd.read_csv(matches_file)
match_ids = all_matches_df['match_id'].unique()[:1100]

# --- Load existing output (if available) ---
if os.path.exists(output_file):
    existing_df = pd.read_csv(output_file)
    processed_matches = set(existing_df['match_id'].unique())
    print(f"Resuming: {len(processed_matches)} matches already processed, {len(existing_df)} rows in file.")
else:
    existing_df = pd.DataFrame()
    processed_matches = set()
    print("No existing file found. Starting fresh.")

# --- Find remaining matches ---
remaining_match_ids = [mid for mid in match_ids if mid not in processed_matches]
print(f"\nPre-Check Summary:")
print(f" → Total match IDs available: {len(match_ids)}")
print(f" → Already processed: {len(processed_matches)} matches")
print(f" → Remaining to process: {len(remaining_match_ids)} matches\n")

def fetch_match_details(match_id):
    url = MATCH_DETAIL_URL.format(match_id=match_id)
    response = requests.get(url)
    if response.status_code == 429:  # Rate limit handling
        retry_after = int(response.headers.get('Retry-After', 60))
        print(f"Rate limit hit. Waiting {retry_after} seconds...")
        time.sleep(retry_after)
        response = requests.get(url)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error fetching match {match_id}: {response.status_code}")
        return None

# --- Fetch matches ---
all_new_data = []
total_rows = len(existing_df)

for i, match_id in enumerate(remaining_match_ids, start=1):
    print(f"Fetching match {i} of {len(remaining_match_ids)} (match_id={match_id})...")
    match_data = fetch_match_details(match_id)
    if match_data:
        player_rows = []
        for player in match_data.get("players", []):
            player_rows.append({
                "match_id": match_id,
                "account_id": player.get("account_id"),
                "hero_id": player.get("hero_id"),
                "kills": player.get("kills"),
                "deaths": player.get("deaths"),
                "assists": player.get("assists"),
                "gold_per_min": player.get("gold_per_min"),
                "xp_per_min": player.get("xp_per_min"),
                "hero_damage": player.get("hero_damage"),
                "hero_healing": player.get("hero_healing"),
                "duration": match_data.get("duration"),
                "radiant_win": match_data.get("radiant_win"),
                "start_time": match_data.get("start_time")
            })
        all_new_data.extend(player_rows)
        total_rows += len(player_rows)
        print(f" → Rows so far: {total_rows}")
    time.sleep(1)

# --- Combine old + new data ---
if len(existing_df) > 0:
    final_df = pd.concat([existing_df, pd.DataFrame(all_new_data)], ignore_index=True)
else:
    final_df = pd.DataFrame(all_new_data)

# --- Save file ---
final_df.to_csv(output_file, index=False)
print(f"\nSaved {len(final_df)} rows to {output_file}")
print(f"Matches processed: {len(final_df['match_id'].unique())} of {len(match_ids)}")

Resuming: 601 matches already processed, 6010 rows in file.

Pre-Check Summary:
 → Total match IDs available: 1100
 → Already processed: 601 matches
 → Remaining to process: 600 matches

Fetching match 1 of 600 (match_id=8391281904)...
 → Rows so far: 6020
Fetching match 2 of 600 (match_id=8391281902)...
 → Rows so far: 6030
Fetching match 3 of 600 (match_id=8391281900)...
 → Rows so far: 6040
Fetching match 4 of 600 (match_id=8391281816)...
 → Rows so far: 6050
Fetching match 5 of 600 (match_id=8391281813)...
 → Rows so far: 6060
Fetching match 6 of 600 (match_id=8391281812)...
 → Rows so far: 6070
Fetching match 7 of 600 (match_id=8391281810)...
 → Rows so far: 6080
Fetching match 8 of 600 (match_id=8391281809)...
 → Rows so far: 6090
Fetching match 9 of 600 (match_id=8391281808)...
 → Rows so far: 6100
Fetching match 10 of 600 (match_id=8391281803)...
 → Rows so far: 6110
Fetching match 11 of 600 (match_id=8391281706)...
 → Rows so far: 6120
Fetching match 12 of 600 (match_id=839128

**Data sanity check** 

Summary of row counts per column, unique players, etc.

In [58]:
import pandas as pd

# Loading dataset
file_path = r"C:\Users\Ellen\Desktop\OpenDota Data\match_details_500matches.csv"
df = pd.read_csv(file_path)

# Summary
print("=== Basic Info ===")
print(f"Rows: {len(df)}")
print(f"Unique matches: {df['match_id'].nunique()}")
print(f"Unique players: {df['account_id'].nunique()}")

print("\n=== Basic Stats ===")
print(df[['kills','deaths','assists','gold_per_min','xp_per_min']].describe())

print("\n=== Hero Frequency ===")
print(df['hero_id'].value_counts().head(10))

=== Basic Info ===
Rows: 12010
Unique matches: 1201
Unique players: 5706

=== Basic Stats ===
              kills        deaths       assists  gold_per_min    xp_per_min
count  12010.000000  12010.000000  12010.000000  12010.000000  12010.000000
mean       7.162032      7.355121     13.385012    465.458118    626.817735
std        5.296589      3.996651      7.740610    149.830950    227.653245
min        0.000000      0.000000      0.000000    103.000000      1.000000
25%        3.000000      4.000000      7.000000    356.000000    465.000000
50%        6.000000      7.000000     12.000000    446.000000    602.000000
75%       10.000000     10.000000     18.000000    568.000000    765.000000
max       37.000000     26.000000     53.000000   1169.000000   1767.000000

=== Hero Frequency ===
hero_id
14     347
2      299
53     278
26     271
27     256
39     251
11     249
30     240
75     232
104    210
Name: count, dtype: int64


**3. Pulling Hero Data**

In [60]:
# OpenDota heroes endpoint
HEROES_URL = "https://api.opendota.com/api/heroes"
response = requests.get(HEROES_URL)

if response.status_code == 200:
    heroes_data = response.json()
    heroes_df = pd.DataFrame(heroes_data)
    print("First 5 heroes:\n", heroes_df.head())

    # Save heroes data
    heroes_path = r"C:\Users\Ellen\Desktop\OpenDota Data\heroes.csv"
    heroes_df.to_csv(heroes_path, index=False)
    print(f"Heroes data saved to {heroes_path}")
else:
    print(f"Failed to fetch heroes: {response.status_code}")

First 5 heroes:
    id                          name  localized_name primary_attr attack_type  \
0   1        npc_dota_hero_antimage       Anti-Mage          agi       Melee   
1   2             npc_dota_hero_axe             Axe          str       Melee   
2   3            npc_dota_hero_bane            Bane          all      Ranged   
3   4     npc_dota_hero_bloodseeker     Bloodseeker          agi       Melee   
4   5  npc_dota_hero_crystal_maiden  Crystal Maiden          int      Ranged   

                                   roles  legs  
0                 [Carry, Escape, Nuker]     2  
1  [Initiator, Durable, Disabler, Carry]     2  
2    [Support, Disabler, Nuker, Durable]     4  
3    [Carry, Disabler, Nuker, Initiator]     2  
4             [Support, Disabler, Nuker]     2  
Heroes data saved to C:\Users\Ellen\Desktop\OpenDota Data\heroes.csv


**Pulling live data (match-level)**

In [65]:
# OpenDota live matches endpoint
LIVE_URL = "https://api.opendota.com/api/live"

def fetch_live_matches():
    response = requests.get(LIVE_URL)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Failed to fetch live matches. Status code: {response.status_code}")
        return None

# Fetch live matches once
live_data = fetch_live_matches()

if live_data:
    # Flatten relevant data for each match
    rows = []
    for match in live_data:
        rows.append({
            "match_id": match.get("match_id"),
            "activate_time": match.get("activate_time"),
            "deactivate_time": match.get("deactivate_time"),
            "league_id": match.get("league_id"),
            "series_id": match.get("series_id"),
            "average_mmr": match.get("average_mmr"),
            "game_mode": match.get("game_mode"),
            "radiant_lead": match.get("radiant_lead"),
            "building_state": match.get("building_state"),
            "radiant_score": match.get("radiant_score"),
            "dire_score": match.get("dire_score"),
            "radiant_team_name": match.get("team_name_radiant"),
            "dire_team_name": match.get("team_name_dire"),
            "radiant_team_id": match.get("team_id_radiant"),
            "dire_team_id": match.get("team_id_dire"),
        })

    df_live = pd.DataFrame(rows)

    # Save to CSV
    output_path = r"C:\Users\Ellen\Desktop\OpenDota Data\live_matches.csv"
    df_live.to_csv(output_path, index=False)
    print(f"Saved {len(df_live)} live matches to {output_path}")
else:
    print("No live matches fetched.")

Saved 100 live matches to C:\Users\Ellen\Desktop\OpenDota Data\live_matches.csv


**Pulling live data (player-level)**

In [70]:
# OpenDota live matches endpoint
LIVE_URL = "https://api.opendota.com/api/live"

def fetch_live_matches():
    response = requests.get(LIVE_URL)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Failed to fetch live matches. Status code: {response.status_code}")
        return None

# Fetch live match data
live_data = fetch_live_matches()

if live_data:
    rows = []
    for match in live_data:
        match_id = match.get("match_id")
        activate_time = match.get("activate_time")
        deactivate_time = match.get("deactivate_time")
        league_id = match.get("league_id")
        series_id = match.get("series_id")
        average_mmr = match.get("average_mmr")
        game_mode = match.get("game_mode")
        radiant_lead = match.get("radiant_lead")
        building_state = match.get("building_state")
        radiant_score = match.get("radiant_score")
        dire_score = match.get("dire_score")
        radiant_team_name = match.get("team_name_radiant")
        dire_team_name = match.get("team_name_dire")
        radiant_team_id = match.get("team_id_radiant")
        dire_team_id = match.get("team_id_dire")

        # Loop through players in each match
        for player in match.get("players", []):
            rows.append({
                "match_id": match_id,
                "activate_time": activate_time,
                "deactivate_time": deactivate_time,
                "league_id": league_id,
                "series_id": series_id,
                "average_mmr": average_mmr,
                "game_mode": game_mode,
                "radiant_lead": radiant_lead,
                "building_state": building_state,
                "radiant_score": radiant_score,
                "dire_score": dire_score,
                "radiant_team_name": radiant_team_name,
                "dire_team_name": dire_team_name,
                "radiant_team_id": radiant_team_id,
                "dire_team_id": dire_team_id,
                "player_account_id": player.get("account_id"),
                "player_name": player.get("name"),
                "hero_id": player.get("hero_id"),
                "team": player.get("team")  # 0 = Radiant, 1 = Dire
            })

    df_live_players = pd.DataFrame(rows)

    # Save to CSV
    output_path = r"C:\Users\Ellen\Desktop\OpenDota Data\live_matches_players.csv"
    df_live_players.to_csv(output_path, index=False)
    print(f"Saved {len(df_live_players)} player rows to {output_path}")
else:
    print("No live matches fetched.")

Saved 975 player rows to C:\Users\Ellen\Desktop\OpenDota Data\live_matches_players.csv


**Sanity Check**

In [75]:
# Load player-level live match data
file_path = r"C:\Users\Ellen\Desktop\OpenDota Data\live_matches_players.csv"
df = pd.read_csv(file_path)

# Basic counts
total_rows = len(df)
unique_matches = df['match_id'].nunique()
unique_players = df['player_account_id'].nunique()

print(f"Rows: {total_rows}")
print(f"Unique matches: {unique_matches}")
print(f"Unique players (public IDs only): {unique_players}")

# Hero frequency
hero_counts = df['hero_id'].value_counts().head(10)
print("\n=== Top 10 Heroes ===")
print(hero_counts)

# Team breakdown
team_counts = df['team'].value_counts()  # 0 = Radiant, 1 = Dire
print("\n=== Team Counts ===")
print(team_counts)


Rows: 975
Unique matches: 98
Unique players (public IDs only): 735

=== Top 10 Heroes ===
hero_id
11     35
53     30
135    29
39     27
14     27
2      27
27     25
86     23
26     21
70     19
Name: count, dtype: int64

=== Team Counts ===
team
1    489
0    486
Name: count, dtype: int64


**4. Pulling Constants Data (items)**

In [78]:
# OpenDota constants items endpoint
ITEMS_URL = "https://api.opendota.com/api/constants/items"

def fetch_items():
    response = requests.get(ITEMS_URL)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Failed to fetch items. Status code: {response.status_code}")
        return None

items_data = fetch_items()

if items_data:
    rows = []
    for key, item in items_data.items():
        rows.append({
            "item_id": item.get("id"),
            "name": key,
            "localized_name": item.get("dname"),
            "cost": item.get("cost"),
            "notes": item.get("notes"),
            "qual": item.get("qual"),
            "created": item.get("created"),
            "charges": item.get("charges")
        })

    df_items = pd.DataFrame(rows)
    output_path = r"C:\Users\Ellen\Desktop\OpenDota Data\items.csv"
    df_items.to_csv(output_path, index=False)
    print(f"Items data saved to {output_path}")
else:
    print("No items data fetched.")

Items data saved to C:\Users\Ellen\Desktop\OpenDota Data\items.csv


**Pulling Constants (game modes)**

In [82]:
# OpenDota constants game modes endpoint
GAMEMODES_URL = "https://api.opendota.com/api/constants/game_mode"

def fetch_game_modes():
    response = requests.get(GAMEMODES_URL)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Failed to fetch game modes. Status code: {response.status_code}")
        return None

game_modes_data = fetch_game_modes()

if game_modes_data:
    rows = []
    for key, gm in game_modes_data.items():
        rows.append({
            "id": gm.get("id"),
            "name": gm.get("name"),
            "balanced": gm.get("balanced")
        })
    df_game_modes = pd.DataFrame(rows)
    output_path = r"C:\Users\Ellen\Desktop\OpenDota Data\game_modes.csv"
    df_game_modes.to_csv(output_path, index=False)
    print(f"Game modes data saved to {output_path}")
else:
    print("No game modes data fetched.")

Game modes data saved to C:\Users\Ellen\Desktop\OpenDota Data\game_modes.csv


**Pulling Constants Data (lobby and region)**

In [84]:
# OpenDota constants endpoints
LOBBY_TYPES_URL = "https://api.opendota.com/api/constants/lobby_type"
REGIONS_URL = "https://api.opendota.com/api/constants/region"

def fetch_constants(url):
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Failed to fetch constants from {url}. Status code: {response.status_code}")
        return None

# --- Fetch Lobby Types ---
lobby_types = fetch_constants(LOBBY_TYPES_URL)
if lobby_types:
    rows = []
    for key, val in lobby_types.items():
        rows.append({
            "id": val.get("id"),
            "name": val.get("name"),
            "description": val.get("description")
        })
    df_lobby_types = pd.DataFrame(rows)
    output_lobby = r"C:\Users\Ellen\Desktop\OpenDota Data\lobby_types.csv"
    df_lobby_types.to_csv(output_lobby, index=False)
    print(f"Lobby types saved to {output_lobby}")
else:
    print("No lobby types data fetched.")

# --- Fetch Regions ---
regions = fetch_constants(REGIONS_URL)
if regions:
    rows = []
    for key, val in regions.items():
        rows.append({
            "region_id": key,
            "region_name": val
        })
    df_regions = pd.DataFrame(rows)
    output_regions = r"C:\Users\Ellen\Desktop\OpenDota Data\regions.csv"
    df_regions.to_csv(output_regions, index=False)
    print(f"Regions saved to {output_regions}")
else:
    print("No regions data fetched.")

Lobby types saved to C:\Users\Ellen\Desktop\OpenDota Data\lobby_types.csv
Regions saved to C:\Users\Ellen\Desktop\OpenDota Data\regions.csv


**6. Pulling Ranking data to (region_id check)**

In [88]:
import requests
import pandas as pd

HERO_ID = 11  # top hero in your live matches
RANKINGS_URL = f"https://api.opendota.com/api/rankings?hero_id={HERO_ID}"

response = requests.get(RANKINGS_URL)
if response.status_code == 200:
    rankings = response.json().get("rankings", [])
    df_rankings = pd.DataFrame(rankings)
    output_path = r"C:\Users\Ellen\Desktop\OpenDota Data\rankings_hero11.csv"
    df_rankings.to_csv(output_path, index=False)
    print(f"Rankings data saved to {output_path}")
    print(df_rankings.head())
else:
    print(f"Failed to fetch rankings. Status code: {response.status_code}")


Rankings data saved to C:\Users\Ellen\Desktop\OpenDota Data\rankings_hero11.csv
   account_id        score  personaname  name  \
0   356090976  8880.315555      гадость  None   
1    33388894  8638.873910          zzz  None   
2   213952287  8567.868860    不上手机令牌封游戏  None   
3   387395868  8555.126059  grafvishnya  None   
4   240842238  8539.649902         Fino  None   

                                              avatar  \
0  https://avatars.steamstatic.com/bb0bd4ceaa58d8...   
1  https://avatars.steamstatic.com/c9fdc165d2e684...   
2  https://avatars.steamstatic.com/44b2254cdd684d...   
3  https://avatars.steamstatic.com/6953b663f4d625...   
4  https://avatars.steamstatic.com/defb3862db8a81...   

                 last_login  rank_tier  
0  2025-05-31T23:26:30.247Z         80  
1  2025-03-25T16:19:33.490Z         80  
2                      None         80  
3  2022-10-26T08:18:19.357Z         80  
4                      None         80  


**7. Pulling Distribution data (region_id check)**

In [91]:
DISTRIBUTION_URL = "https://api.opendota.com/api/distributions"

response = requests.get(DISTRIBUTION_URL)
if response.status_code == 200:
    data = response.json()
    # Check what keys are present
    print("Keys returned:", data.keys())

    # Save full JSON to file for inspection
    output_path = r"C:\Users\Ellen\Desktop\OpenDota Data\distribution_full.json"
    import json
    with open(output_path, "w") as f:
        json.dump(data, f, indent=4)
    print(f"Distribution data saved to {output_path}")
else:
    print(f"Failed to fetch distribution data. Status code: {response.status_code}")

Keys returned: dict_keys(['ranks'])
Distribution data saved to C:\Users\Ellen\Desktop\OpenDota Data\distribution_full.json


**Converting distribution from json to csv**

In [96]:
# --- Fetch distribution data ---
DISTRIBUTION_URL = "https://api.opendota.com/api/distributions"
response = requests.get(DISTRIBUTION_URL)

if response.status_code == 200:
    data = response.json()

    # Extract the ranks rows
    ranks_data = data.get("ranks", {}).get("rows", [])
    df = pd.DataFrame(ranks_data)

    # Save to CSV
    output_path = r"C:\Users\Ellen\Desktop\OpenDota Data\rank_distributions.csv"
    df.to_csv(output_path, index=False)
    print(f"Rank distributions saved to {output_path}")
    print(df.head())
else:
    print(f"Failed to fetch distribution data. Status code: {response.status_code}")

Rank distributions saved to C:\Users\Ellen\Desktop\OpenDota Data\rank_distributions.csv
   bin  bin_name   count  cumulative_sum
0   11        11    4460            4460
1   12        12   79936           84396
2   13        13   90740          175136
3   14        14  115380          290516
4   15        15  133126          423642
